In [4]:
import math
from statistics import NormalDist


# Option Pricing Exercises

## Black Scholes Option Pricing Formula

[Black and Scholes (1973)](https://www.cs.princeton.edu/courses/archive/fall09/cos323/papers/black_scholes73.pdf) derive a formula that can be used to value a *European* option that does **not** pay dividends before the option's expiration 
date:

Let 

+ $S$ = Stock Price
+ $X$ = Strike/Exercise Price
+ $r$ = Risk Free Rate 
+ $T$ = Time to expiration in years
+ $\sigma$ = Volatility of the relative price change of the underlying stock price
+ $N(x)$ = The [cumulative normal distribution function](https://statproofbook.github.io/P/norm-cdf.html)

Then
$$c = SN(d_{1})-Xe^{-rT}N(d_{2})$$
$$p = Xe^{-rT}N(-d_{2})-SN(-d_{1})$$

Where


$$d_{1} = \frac{ln(\frac{S}{X}) + (\frac{r+\sigma^{2}}{2})T}{\sigma\sqrt{T}}$$
$$d_{2} = \frac{ln(\frac{S}{X}) + (\frac{r-\sigma^{2}}{2})T}{\sigma\sqrt{T}} = d_{1} -\sigma\sqrt{T}$$



In [5]:
# Formula

def price_bs_euro_option_(S, X, r, sigma, T, type='c'):

    d1 = (math.log(S/X) + (r + sigma**2)/2*T) / sigma * T**0.5
    d2 = d1 - sigma * T**0.5

    if type == 'c':
        return S * NormalDist().cdf(d1) - X * math.exp(-r*T) * NormalDist(d2)

    elif type == 'p':
        return X * math.exp(-r*T) * NormalDist(-d2) - S * NormalDist(-d1)

# Alternatively
def price_bs_euro_option(spot, strike, rf_rate, vol, ttmy, type='c'):

    d1 = (math.log(spot/strike) + (rf_rate + vol**2/2)*ttmy) / (vol*ttmy**0.5)
    d2 = d1 - vol * ttmy**0.5

    if type == 'c':

        return spot * NormalDist().cdf(x=d1) - strike * math.exp(-rf_rate*ttmy) * NormalDist().cdf(x=d2)

    elif type == 'p':

        return strike * math.exp(-rf_rate*ttmy) * NormalDist().cdf(-d2) - spot * NormalDist().cdf(-d1)


Let's consider the following scenario:

- European option 
- 3 months to expiry, therfore $\frac{3}{12} = 0.25$ because we measure in years
- Stock(spot) price = R60.00
- Strike price = R65.00
- Risk free rate = 8% p.a.
- Volatility = 30%

This is calculated as follows:

In [6]:
price_bs_euro_option(spot=60, strike=65, rf_rate=0.08,
                  ttmy=0.25, vol=0.3, type='c')


2.133368444916197

## Options on Stock Indices

[Merton (1973)](http://polymer.bu.edu/hes/merton73py538.pdf) extended the Black-Scholes model to allow for a continuous dividend yield. This model can price European put/call options on a stock (or stock index) paying a known dividend yield, $q$. This implies the addition of a $e^{-qT}$ term to the put and call equations from earlier:

$$c = Se^{-qT}N(d_{1})-Xe^{-rT}N(d_{2})$$
$$p = Xe^{-rT}N(-d_{2})-Se^{-qT}N(-d_{1})$$
Where $d_1$ and $d_2$ are now

$$d_{1} = \frac{ln(\frac{S}{X}) + (r-q+\frac{\sigma^{2}}{2})T}{\sigma\sqrt{T}}$$

$$d_{2} = \frac{ln(\frac{S}{X}) + (r-q-\frac{\sigma^{2}}{2})T}{\sigma\sqrt{T}} = d_{1} -\sigma\sqrt{T}$$

In [7]:
def price_bsm_euro_option(spot, strike, rf_rate, vol, ttmy, div_yield, type='c'):

    d1 = (math.log(spot/strike) + (rf_rate - div_yield   + vol**2/2)*ttmy) / (vol*ttmy**0.5)
    d2 = d1 - vol * ttmy**0.5

    if type == 'c':

        return spot * math.exp(-div_yield*ttmy) * NormalDist().cdf(x=d1) - strike * math.exp(-rf_rate*ttmy) * NormalDist().cdf(x=d2)

    elif type == 'p':

        return strike * math.exp(-rf_rate*ttmy) * NormalDist().cdf(-d2) - spot * math.exp(-div_yield*ttmy) * NormalDist().cdf(-d1)

Let's consider the following scenario:

- European option 
- 6 months to expiry, therefore $\frac{6}{12} = 0.5$ because we measure in years
- Stock(spot) price = R100.00
- Strike price = R95.00
- Risk free rate = 10% p.a.
- Volatility = 20% p.a.

This is calculated as follows:

In [8]:
price_bsm_euro_option(
  spot = 100,
  strike = 95,
  rf_rate = 0.1,
  ttmy = 0.5,
  div_yield = 0.05,
  vol = 0.2,
  type = "p"
)

2.4647876467558234

## Options on Futures

The formula from [Black (1973)](https://www.sciencedirect.com/science/article/abs/pii/0304405X76900246) provides the price of a European option when the underlying security is a forward or futures contract with an initial price, $F$:

$$c = e^{-rT}[FN(d_{1})-XN(d_{2})]$$
$$p = e^{-rT}[XN(-d_{2})-FN(-d_{1})]$$
Where

$$d_{1} = \frac{ln(\frac{F}{X}) + (\frac{\sigma^{2}}{2})T}{\sigma\sqrt{T}}$$
$$d_{2} = \frac{ln(\frac{F}{X}) - (\frac{\sigma^{2}}{2})T}{\sigma\sqrt{T}} = d_{1} -\sigma\sqrt{T}$$

In [9]:
def price_futures_euro_option(spot, strike, rf_rate, vol, ttmy, type='c'):

    d1 = (math.log(spot/strike) + (vol**2/2)*ttmy) / (vol*ttmy**0.5)
    d2 = d1 - vol * ttmy**0.5

    if type == 'c':

        return math.exp(-rf_rate*ttmy) * (spot * NormalDist().cdf(x=d1) - strike * NormalDist().cdf(x=d2))

    elif type == 'p':

        return math.exp(-rf_rate*ttmy) * (strike * NormalDist().cdf(-d2) - spot * NormalDist().cdf(-d1))

Let's consider the following scenario:

- A European _call_ option on [Brent Blend](https://www.investopedia.com/terms/b/brentblend.asp) futures
- 9 months to expiry, therefore $\frac{9}{12} = 0.75$ because we measure in years
- Futures(spot) price = R19.00
- Strike price = R19.00
- Risk free rate = 10% p.a.
- Volatility = 28% p.a.

This is calculated as follows:

In [11]:
price_futures_euro_option(
  spot = 19,
  strike = 19,
  rf_rate = 0.1,
  ttmy = 0.75,
  vol = 0.28,
  type = "c"
)

1.701050725236268

In [12]:
price_futures_euro_option(
  spot = 19,
  strike = 19,
  rf_rate = 0.1,
  ttmy = 0.75,
  vol = 0.28,
  type = "p"
)

1.701050725236268